In [1]:
import pandas as pd
from thefuzz import fuzz

In [2]:
df = pd.read_excel('evictions_2023.xlsx')

In [3]:
print(f'Original Dataframe Length: {df.shape[0]}')

Original Dataframe Length: 16657


In [4]:
# get relevant columns and rename for clarity
df = df[['CaseNumber',
         'CaseFileDate',
         'CITY_NAME',
         'POSTAL_CD',
         'Disposition']]

df = df.rename(columns={'CaseNumber'   : 'case_number',
                        'CaseFileDate' : 'file_date',
                        'CITY_NAME'    : 'city',
                        'POSTAL_CD'    : 'zip_code',
                        'Disposition'  : 'disposition'})

In [5]:
# Drop rows where cases were dismissed

df['dismissed'] = df.disposition.apply(lambda x : 'Dismissed' in x)

df = df[df.dismissed == False]

print(f'Dataframe Length after Dropping Dismissed Cases: {df.shape[0]}')

Dataframe Length after Dropping Dismissed Cases: 16576


In [6]:
df[df.case_number == '31E2304740']

,case_number,file_date,city,zip_code,disposition,dismissed
8793,31E2304740,2023-12-08,San Antonio,78259,Appealed,False
8794,31E2304740,2023-12-08,San Antonio,78259,Default Judgments (OCA),False
8795,31E2304740,2023-12-08,San Antonio,78259,Judgment for Plaintiff (OCA),False
8796,31E2304740,2023-12-08,San Antonio,78259,Appealed,False
8797,31E2304740,2023-12-08,San Antonio,78259,Default Judgments (OCA),False
8798,31E2304740,2023-12-08,San Antonio,78259,Judgment for Plaintiff (OCA),False


In [ ]:
df = df.drop_duplicates()

print(f'Dataframe Length after Dropping Duplicates cases: {df.shape[0]}')

In [ ]:
# adjust city values for misspellings of san antonio and lowercase all values

def fuzzy_sa(value):
    ''' takes in a pandas value and returns san antonio if string is a 90 or higher ratio to san antonio
        otherwise returns lower cased version of the original string'''
    
    value = str(value).lower()
    
    sa1 = 'san antonio'
    sa2 = 'sa'
    
    ratio1 = fuzz.ratio(sa1,value)
    ratio2 = fuzz.ratio(sa2,value)
    
    if (ratio1 >= 80) or (ratio2 >= 60):
        
        return 'san antonio'
    
    else:
        
        return value


df['city'] = df.city.apply(fuzzy_sa)

In [ ]:
for col in df.columns:
    
    print (col)
    print(df[f'{col}'].value_counts().head(20))
    print()

In [ ]:
df.duplicated().sum()

In [ ]:
df.head()

In [ ]:


for col in df.columns:
    
    print (col)
    print(df[f'{col}'].value_counts())
    print()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df[df.DefendantAddress.isnull()]

In [ ]:
df.CITY_NAME.value_counts()

In [ ]:
df.Pl_State.value_counts()

In [ ]:
from thefuzz import fuzz, process

In [ ]:
s1 = "San Antonio".lower()
s2 = "SAN ANTONI".lower()

print(fuzz.ratio(s1,s2))

In [ ]:
def fuzzy_sa(value):
    ''' takes in a pandas value and returns san antonio if string is a 90 or higher ratio to san antonio
        otherwise returns lower cased version of the original string'''
    
    sa = 'san antonio'
    value = value.lower()
    ratio = fuzz.ratio(sa,value)
    
    if ratio >= 90:
        
        return 'san antonio'
    
    else return value
    
    

In [ ]:
fuzzy_sa('SAN ANTONI')